In [118]:
import typing


FilePath: typing.TypeAlias = str
FileContent: typing.TypeAlias = list[str]


class FileReader(typing.Protocol):
    def __call__(self, file: FilePath) -> FileContent: ...


def file_reader(file: FilePath) -> FileContent:
    content: FileContent
    with open(file) as file_handler:
        return [line for line in file_handler.read().splitlines(keepends=False) if len(line) > 0]


data_reader: FileReader = file_reader

In [119]:
from rich import print

garden_plots: FileContent = data_reader("../media/2024-day-12.input")

In [120]:
import uuid
from dataclasses import dataclass, field, asdict


@dataclass(frozen=True)
class Point:
    x: int
    y: int
    plant_type: str


@dataclass(frozen=True)
class Neighbours:
    left: Point | None
    right: Point | None
    up: Point | None
    down: Point | None


@dataclass
class Region:
    _id: uuid.UUID = field(init=False)
    points: list[Point] = field(default_factory=list)
    area: int | None = None
    perimeter: int | None = None
    price: int | None = None

    def __post_init__(self):
        self._id = uuid.uuid4()

    def calculate_area(self) -> int:
        self.area = len(self.points)
        return self.area        

    def calculate_price(self) -> int:
        if self.perimeter is None:
            raise ValueError("Perimetr is missing for region: ", self._id)
        area: int = self.calculate_area()
        self.price = area * self.perimeter
        return self.price


@dataclass
class Grid:
    content: FileContent = field(default_factory=list)
    data: list[list[Point | None]] = field(default_factory=list)
    regions: list[Region] = field(default_factory=list)    

    def load_from_file(self, file_content: FileContent):
        self.content = file_content
        self.data = []
        for y, line in enumerate(file_content):
            for x, character in enumerate(line):
                if x == 0:
                    self.data.append([Point(x=x, y=y, plant_type=character)])
                else:
                    self.data[y].append(Point(x=x, y=y, plant_type=character))

    def get_point(self, x: int, y: int) -> Point:
        return self.data[y][x]

    def get_neighbours(self, point: Point | None, region: bool = False) -> Neighbours | set[Point]:
        def region_filter(_point: Point | None) -> Point | None:
            if region and _point and _point.plant_type != point.plant_type:
                return None
            return _point
        if point is None:
            if region:
                return set()
            else:
                return Neighbours(left=None, right=None, up=None, down=None)
        left, right, up, down = None, None, None, None 
        if point.x > 0:
            left = region_filter(self.data[point.y][point.x-1])
        if point.x < len(self.data[point.y]) - 1:
            right = region_filter(self.data[point.y][point.x+1])
        if point.y > 0:
            up = region_filter(self.data[point.y-1][point.x])
        if point.y < len(self.data) - 1:
            down = region_filter(self.data[point.y+1][point.x])
        if region:
            return set(p for p in [left, right, up, down] if p != None)
        else:
            return Neighbours(left=left, right=right, up=up, down=down)

    def get_perimeter(self, point: Point) -> int:
        neighbours: Neighbours = self.get_neighbours(point)
        perimeter: int = 0
        for _point in asdict(neighbours).values():
            if not _point or _point["plant_type"] != point.plant_type:
                perimeter += 1
        return perimeter

    def calculate_perimeter(self) -> int:
        perimeter: int = 0
        for region in self.regions:
            _perimeter: int = 0
            for point in region.points:
                _perimeter += self.get_perimeter(point)
            region.perimeter = _perimeter
            perimeter += _perimeter
        return perimeter

    def calculate_price(self) -> int:
        price: int = 0
        for region in self.regions:
            perimeter: int = 0
            for point in region.points:
                perimeter += self.get_perimeter(point)
            region.perimeter = perimeter
            price += region.calculate_price()
        return price

    def find_region(self, point: Point) -> Region | None:
        for region in filter(lambda region: point in region.plots, self.regions):
            yield region
        return None

    def reveal_regions(self):
        if self.regions:
            return
        for y, points in enumerate(self.data):
            for x, point in enumerate(points):
                if point is None:
                    continue
                neighbours: set[Point] = set()
                _neighbours: set[Point] = set((point,))
                while _neighbours:
                    _next = set()
                    for neighbour in _neighbours:
                        self.data[neighbour.y][neighbour.x] = None
                        _next.update(self.get_neighbours(neighbour, region=True))
                    neighbours.update(_neighbours)
                    _neighbours = _next
                self.regions.append(Region(points=neighbours))
        self.load_from_file(self.content)

In [121]:
grid: Grid = Grid()
grid.load_from_file(garden_plots)

In [122]:
point = grid.get_point(2,1)
neighbours = grid.get_neighbours(point)
region_neighbours = grid.get_neighbours(point, region=True)
perimeter = grid.get_perimeter(point)

In [123]:
print(point)
print(neighbours)
print("Region neighbours: ", region_neighbours)
print("Perimeter: ", perimeter)

Point(x=2, y=1, plant_type='N')

Neighbours(
    left=Point(x=1, y=1, plant_type='N'),
    right=Point(x=3, y=1, plant_type='N'),
    up=Point(x=2, y=0, plant_type='N'),
    down=Point(x=2, y=2, plant_type='N')
)

Region neighbours: 
{
    Point(x=3, y=1, plant_type='N'),
    Point(x=1, y=1, plant_type='N'),
    Point(x=2, y=0, plant_type='N'),
    Point(x=2, y=2, plant_type='N')
}

Perimeter:  0

In [124]:
grid.reveal_regions()
print("Grid perimeter: ", grid.calculate_perimeter())
print("Grid price: ", grid.calculate_price())

Grid perimeter:  17068

Grid price:  1473276